# Experiment Documentation: Identification in N:M Search Spaces

This experiment investigates the dynamics of feature correspondences within an N:M matching framework, leveraging comparisons between original and transformed images. The evaluation considers three detectors: our method, REKD, and KeyNet, alongside two established descriptors: SIFT and HardNet. The study focuses on understanding the interplay between these components and their influence on image identification across datasets of varying complexity: Flowers, Woods, and UV Fibers from Banknotes.

## Methodology

To ensure comprehensive analysis, an **N:M matching process** is applied to evaluate feature correspondences. The selected datasets present progressively increasing challenges:

**Flowers**: Patterns characterized by repetition and detail.

**Woods**: A blend of structural and textural features.

**UV Fibers from Banknotes**: Complex micro-patterns with high forensic relevance.

This methodology facilitates a granular assessment of the robustness and reliability of detector-descriptor combinations under diverse conditions.

## Objective

The objective of this experiment is to analyze how the careful selection of robust keypoints and descriptors enhances the efficacy of image identification pipelines. By addressing the challenges posed by datasets of escalating complexity, this study offers critical insights into optimizing detection and matching strategies for advanced real-world applications.

In [1]:
import torch
import numpy as np
import kornia
from kornia.feature import DescriptorMatcher

class LocalComparisonPipeline:
    """
    Compara imagens de inspeção e referência usando características locais e correspondência de descritores.
    """
    def __init__(self, local_feature, descriptor_matcher):
        self.local_feature = local_feature
        self.descriptor_matcher = descriptor_matcher

    def run(self, inspection_images: torch.Tensor, reference_images: torch.Tensor) -> np.ndarray:
        """
        Compara imagens de inspeção e referência e retorna uma matriz de pontuação baseada em correspondências de descritores.
        """
        n, m = inspection_images.shape[0], reference_images.shape[0]
        scores = np.zeros((n, m))
        cache_reference = {}

        for i_index, i_image in enumerate(inspection_images):
            lafs0, _, descriptors0 = self.local_feature(i_image[:1][None])

            for r_index, r_image in enumerate(reference_images):
                if r_index not in cache_reference:
                    _, _, descriptors1 = self.local_feature(r_image[:1][None])
                    cache_reference[r_index] = descriptors1
                descriptors1 = cache_reference[r_index]

                _, matches = self.descriptor_matcher(descriptors0[0], descriptors1[0])

                num_match = matches.shape[0]
                scores[i_index, r_index] = num_match if num_match >= 3 else 0

        return scores

    def evaluate_matches(self, matches_matrix, threshold=0.5):
        """
        Avalia correspondências entre imagens e calcula TP, FP e FN com base no limiar de similaridade.
        """
        n, m = matches_matrix.shape
        TP, FP, FN = 0, 0, 0

        for i in range(m):
            if np.max(matches_matrix[i]) >= threshold and np.argmax(matches_matrix[i]) == i:
                TP += 1
            else:
                FN += 1

        for i in range(m, n):
            if np.max(matches_matrix[i]) >= threshold and np.argmax(matches_matrix[i]) < m:
                FP += 1

        return TP, FP, FN


In [2]:
class AugmentationGenerator:
    def __init__(self, n_variations):
        # Definir as augmentações
        aug_gen = kornia.augmentation.AugmentationSequential(
            kornia.augmentation.RandomAffine(degrees=360, translate=(0.25, 0.25), scale=(0.95, 1.05), shear=10, p=0.85),
            kornia.augmentation.RandomPerspective(0.3, p=0.85),
            kornia.augmentation.RandomBoxBlur((3, 3), p=0.85),
            data_keys=["input"],
            same_on_batch=True,
        )


        self.augmentation_sequence = aug_gen
        self.n_variations = n_variations
        self.param_list = []
        self.current_index = 0

    def generate_variations(self, input_tensor):
        """
        Gera múltiplas variações de augmentações e coleta seus parâmetros.
        """
        for _ in range(self.n_variations):
            self.augmentation_sequence(input_tensor)            
            self.param_list.append(self.augmentation_sequence._params)

    def __iter__(self):
        self.current_index = 0  # Resetar o índice a cada nova iteração
        return self

    def __next__(self):
        """
        Retorna a próxima variação de parâmetros de augmentação.
        A iteração será circular.
        """
        result = self.param_list[self.current_index]
        self.current_index = (self.current_index + 1) % len(self.param_list)
        return result
    
    def reset(self):
        """Método para resetar o estado do gerador de augmentação."""
        self.current_index = 0  # Reseta o índice de iteração


In [3]:
import torch
import kornia as K
import logging
from itertools import product
from teste_util import read_dataload_woods,read_dataload_fibers,read_dataload_flower,set_seed, check_and_clear_memory
from LocalFeatureCombinations import KeyNetFeatureSIFT,KeyNetFeatureSosNet,REKDSosNet,SingularPointSosNet,REKDHardNet, SingularPointHardNet
from utils.TensorImgIO import imshow_points
import warnings
warnings.filterwarnings("ignore")

# Logger específico
logger = logging.getLogger("Main")  # Logger para a execução principal
logger.setLevel(logging.INFO)  # Define o nível de log como INFO

# Definição do grid de parâmetros
param_grid = {
    'num_features': [60],  # Número de características
    'feature_local': [SingularPointHardNet],  # KeyNetFeatureSIFT,KeyNetFeatureSosNet,REKDSosNet,REKDHardNet,SingularPointSosNet,SingularPointHardNet
    'distance': [0.9],  # Distância para a correspondência (padrão 0.8)
    'threshold': [5]  # Limite para correspondência
}

# Função para configurar o detector e executar o código para uma combinação de parâmetros
def run_experiment(dataloader, pipeline,threshold,aug_gen):
    TP_total, FP_total, FN_total = 0, 0, 0  # Inicializa os contadores totais de TP, FP e FN

    for inspection_batch, _ in dataloader:
        inspection_batch = inspection_batch.to(device)        
        params_item = next(aug_gen)
        reference_batch_t =aug_gen.augmentation_sequence(inspection_batch,params=params_item)
        # Pega apenas a metade inicial de reference_batch_t
        half_reference_batch = reference_batch_t[:reference_batch_t.shape[0] // 2]
        logger.debug(f"Lote inspecao: {inspection_batch.shape} Lote referencia {half_reference_batch.shape}")

        # Executa o pipeline de detecção de características
        with torch.no_grad():
            scores = pipeline.run(inspection_batch, half_reference_batch)
            TP, FP, FN = pipeline.evaluate_matches(scores,threshold=threshold)

            # Acumula os resultados de TP, FP e FN
            TP_total += TP
            FP_total += FP
            FN_total += FN
            
            logger.debug(f"TP: {TP}, FP: {FP}, FN: {FN}")
    
    # Exibe os resultados totais de TP, FP e FN após o experimento
    logger.debug(f"Total de TP: {TP_total}, FP: {FP_total}, FN: {FN_total}")
    return TP_total, FP_total, FN_total

if __name__ == "__main__":
    set_seed(42)
    # Inicializa o dispositivo (GPU ou CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logger.info(f"Dispositivo: {device}")
    # 
    # args = get_config_singular(jupyter=True)

    trainloader, testloader = read_dataload_woods(120,batch_size=132)
    # trainloader, testloader = read_dataload_flower(120,batch_size=132)
    # trainloader, testloader = read_dataload_fibers(120,batch_size=132)
    # gerar variacao de transformacoes pespectivas e fotometrica
    iterator = iter(testloader)
    img, labels = next(iterator)
    aug_gen = AugmentationGenerator(10)
    aug_gen.generate_variations(img)

    # Itera entre todas as combinações de parâmetros
    for num_features, feature_local_class, distance, threshold in product(*param_grid.values()):
        logger.info(f"Executando experimento com: {num_features} características, "
            f"detector {feature_local_class.__name__}, "
            f"distância {distance}, e limiar {threshold}")
        set_seed(1)
    
        # Executa o pipeline de detecção de características
        with torch.no_grad():
            # Inicializa o detector com a configuração
            local_feature = feature_local_class(num_features=num_features, device=device).to(device).eval()
            descriptor_matcher = DescriptorMatcher(match_mode="snn", th=distance).to(device).eval()  # Usando o matcher SNN
            pipeline = LocalComparisonPipeline(local_feature=local_feature, descriptor_matcher=descriptor_matcher)
        
            # Executa o experimento e retorna os totais de TP, FP e FN
            TP_total, FP_total, FN_total = run_experiment(testloader, pipeline,threshold, aug_gen)
            aug_gen.reset()# //TODO: reiniciar o gerador de transformacoes
            # //FIXME: o TP_total e FN_total devem ser 0
            check_and_clear_memory()
            # Calcula a precisão, recall e F1-score
            precision = TP_total / (TP_total + FP_total) if (TP_total + FP_total) > 0 else 0
            recall = TP_total / (TP_total + FN_total) if (TP_total + FN_total) > 0 else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            logger.info(f"TP: {TP_total}, FP: {FP_total}, FN: {FN_total}")
            logger.info(f"Precisão: {precision:.2f}, Recall: {recall:.2f}, F1-score: {f1_score:.2f}")

INFO:Main:Dispositivo: cuda


train True   598
train False   7946
7920


INFO:Main:Executando experimento com: 60 características, detector SingularPointHardNet, distância 0.9, e limiar 5
INFO:Main:TP: 3893, FP: 2027, FN: 67
INFO:Main:Precisão: 0.66, Recall: 0.98, F1-score: 0.79


aug_gen = kornia.augmentation.AugmentationSequential(
    kornia.augmentation.RandomAffine(degrees=360, translate=(0.25, 0.25), scale=(0.95, 1.05), shear=10, p=0.85),
    kornia.augmentation.RandomPerspective(0.3, p=0.85),
    kornia.augmentation.RandomBoxBlur((3, 3), p=0.85),
    data_keys=["input"],
    same_on_batch=True,
)


FLOWERS
1020

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 5
INFO:Main:TP: 259, FP: 510, FN: 251
INFO:Main:Precisão: 0.34, Recall: 0.51, F1-score: 0.41
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 293, FP: 122, FN: 217
INFO:Main:Precisão: 0.71, Recall: 0.57, F1-score: 0.63
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 506, FP: 171, FN: 4
INFO:Main:Precisão: 0.75, Recall: 0.99, F1-score: 0.85
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 510, FP: 165, FN: 0
INFO:Main:Precisão: 0.76, Recall: 1.00, F1-score: 0.86

6072
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 5
INFO:Main:TP: 1358, FP: 3036, FN: 1678
INFO:Main:Precisão: 0.31, Recall: 0.45, F1-score: 0.37
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 1560, FP: 1243, FN: 1476
INFO:Main:Precisão: 0.56, Recall: 0.51, F1-score: 0.53
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 3027, FP: 1619, FN: 9
INFO:Main:Precisão: 0.65, Recall: 1.00, F1-score: 0.79
INFO:Main:Executando experimento com: 60 características, detector REKDHardNet, distância 0.9, e limiar 5
INFO:Main:TP: 3017, FP: 2493, FN: 19
INFO:Main:Precisão: 0.55, Recall: 0.99, F1-score: 0.71
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 3035, FP: 1645, FN: 1
INFO:Main:Precisão: 0.65, Recall: 1.00, F1-score: 0.79
INFO:Main:Executando experimento com: 60 características, detector SingularPointHardNet, distância 0.9, e limiar 5
INFO:Main:TP: 3033, FP: 2459, FN: 3
INFO:Main:Precisão: 0.55, Recall: 1.00, F1-score: 0.71




WOODS

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 5
INFO:Main:TP: 567, FP: 3947, FN: 3393
INFO:Main:Precisão: 0.13, Recall: 0.14, F1-score: 0.13
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 2141, FP: 1976, FN: 1819
INFO:Main:Precisão: 0.52, Recall: 0.54, F1-score: 0.53
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 2670, FP: 2858, FN: 1290
INFO:Main:Precisão: 0.48, Recall: 0.67, F1-score: 0.56
INFO:Main:Executando experimento com: 60 características, detector REKDHardNet, distância 0.9, e limiar 5
INFO:Main:TP: 2604, FP: 3185, FN: 1356
INFO:Main:Precisão: 0.45, Recall: 0.66, F1-score: 0.53
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 3862, FP: 1141, FN: 98
INFO:Main:Precisão: 0.77, Recall: 0.98, F1-score: 0.86
INFO:Main:Executando experimento com: 60 características, detector SingularPointHardNet, distância 0.9, e limiar 5
INFO:Main:TP: 3893, FP: 2027, FN: 67
INFO:Main:Precisão: 0.66, Recall: 0.98, F1-score: 0.79

FIBERS

INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSIFT, distância 0.9, e limiar 5
INFO:Main:TP: 24, FP: 198, FN: 174
INFO:Main:Precisão: 0.11, Recall: 0.12, F1-score: 0.11
INFO:Main:Executando experimento com: 60 características, detector KeyNetFeatureSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 9, FP: 198, FN: 189
INFO:Main:Precisão: 0.04, Recall: 0.05, F1-score: 0.04
INFO:Main:Executando experimento com: 60 características, detector REKDSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 2, FP: 21, FN: 196
INFO:Main:Precisão: 0.09, Recall: 0.01, F1-score: 0.02
INFO:Main:Executando experimento com: 60 características, detector REKDHardNet, distância 0.9, e limiar 5
INFO:Main:TP: 3, FP: 24, FN: 195
INFO:Main:Precisão: 0.11, Recall: 0.02, F1-score: 0.03
INFO:Main:Executando experimento com: 60 características, detector SingularPointSosNet, distância 0.9, e limiar 5
INFO:Main:TP: 122, FP: 56, FN: 76
INFO:Main:Precisão: 0.69, Recall: 0.62, F1-score: 0.65
INFO:Main:Executando experimento com: 60 características, detector SingularPointHardNet, distância 0.9, e limiar 5
INFO:Main:TP: 126, FP: 101, FN: 72
INFO:Main:Precisão: 0.56, Recall: 0.64, F1-score: 0.59